In [10]:
import csv

filename = "data/ODI-2018.csv"

with open(filename,'r') as csvfile:
    # Set the right quote character and delimiter
    csv_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

print(csv_contents[3])

{'What programme are you in?': 'Business Analytics', 'When is your birthday (date)?': '25 december 92', 'Did you stand up?': 'no', 'What is your gender?': 'male', 'Have you taken a course on information retrieval?': '1', 'Chocolate makes you.....': 'neither', 'Have you taken a course on statistics?': 'sigma', 'Time you went to be Yesterday': '23', 'Have you taken a course on machine learning?': 'yes', 'Number of neighbors sitting around you?': '5', 'What makes a good day for you (2)?': 'Sport', 'Give a random number': '6', 'Timestamp': '4/5/2018 11:23:06', 'Have you taken a course on databases?': 'ja', 'You can get £100 if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would get then? ': 'Not enough', 'What makes a good day for you (1)?': 'Food'}
